<a href="https://colab.research.google.com/github/NzimbaEnvoy/Fraud_Detection-Masters-Project-/blob/main/E_Commerce_Fraud_Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Importing Libraries
import os
import pandas as pd
import numpy as np
import time
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.metrics import Precision, Recall
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve, accuracy_score, precision_score, recall_score, f1_score
from imblearn.over_sampling import SMOTE, ADASYN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, LSTM, TimeDistributed, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE, ADASYN
from sklearn.utils.class_weight import compute_class_weight
import itertools
import shap


In [ ]:
# Path to the files
base_path = "/content/drive/MyDrive/Thesis Project"

train_path = os.path.join(base_path, "train_data_E-commerce.csv")
test_path = os.path.join(base_path, "test_data_E-commerce.csv")

# Load data
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

train_df.head(5)

In [ ]:
X_train = train_df.drop("isFraud", axis=1)
y_train = train_df["isFraud"]
X_test = test_df.drop("isFraud", axis=1)
y_test = test_df["isFraud"]

**Baseline CNN**

In [ ]:
# Reshaping for CNN input
X_train_cnn = X_train.values.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test_cnn  = X_test.values.reshape((X_test.shape[0], X_test.shape[1], 1))

In [ ]:
# Early stopping
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True,
    verbose=1
)
#CNN architecture
cnn_baseline_isfraud = Sequential([
    # Block 1
    Conv1D(32, 3, activation='relu', padding='same', input_shape=(X_train_cnn.shape[1], 1)),
    BatchNormalization(),
    Conv1D(64, 3, activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),

    # Block 2
    Conv1D(128, 3, activation='relu', padding='same'),
    BatchNormalization(),
    Conv1D(128, 3, activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),

    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.4),
    Dense(1, activation='sigmoid')
])

In [ ]:
# Compiling
cnn_baseline_isfraud.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy',
                             metrics=['accuracy', Precision(name='precision'), Recall(name='recall')])

# Fitting model
cnn_baseline_isfraud.fit(
    X_train_cnn, y_train,
    epochs=30,
    batch_size=64,
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=1
)

# Predicting and evaluation
y_pred_probs_baseline_cnn_isfraud = cnn_baseline_isfraud.predict(X_test_cnn).flatten()
y_pred_baseline_cnn_isfraud = (y_pred_probs_baseline_cnn_isfraud > 0.5).astype(int)

cm_baseline_cnn_isfraud = confusion_matrix(y_test, y_pred_baseline_cnn_isfraud)
roc_baseline_cnn_isfraud = roc_auc_score(y_test, y_pred_probs_baseline_cnn_isfraud)
cm_baseline_cnn_isfraud

SHAP ANALYSIS FOR CNN BASELINE

In [ ]:
#Extracting Feature Names
feature_names = X_train.columns.tolist()

#Sample background and test data
background = X_train_cnn[np.random.choice(X_train_cnn.shape[0], 200, replace=False)]
X_explain = X_test_cnn[np.random.choice(X_test_cnn.shape[0], 100, replace=False)]

#Flatten 3D tensors to 2D
background_flat = background.reshape(background.shape[0], -1)
X_explain_flat = X_explain.reshape(X_explain.shape[0], -1)

#Define CNN prediction wrapper
def cnn_isfraud_predict(x_flat):
    x_reshaped = x_flat.reshape((-1, X_train_cnn.shape[1], 1))
    return cnn_baseline_isfraud.predict(x_reshaped).flatten()

#Create SHAP KernelExplainer
explainer = shap.KernelExplainer(cnn_isfraud_predict, background_flat)

#Compute SHAP values
shap_values = explainer.shap_values(X_explain_flat)

In [ ]:
#SHAP Summary Plot
plt.figure(figsize=(6, 6))
shap.summary_plot(
    shap_values,
    X_explain_flat,
    feature_names=feature_names,
    plot_type="dot"
)
plt.title("SHAP Summary Plot - CNN (Baseline/No Class Balance)", fontsize=10)
plt.tight_layout()



SMOTE + CNN

In [ ]:
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
X_train_smote_cnn = X_train_smote.values.reshape((X_train_smote.shape[0], X_train_smote.shape[1], 1))

# Early stopping
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True,
    verbose=1
)

#CNN architecture
cnn_smote_isfraud = Sequential([
    # Block 1
    Conv1D(32, 3, activation='relu', padding='same', input_shape=(X_train_smote_cnn.shape[1], 1)),
    BatchNormalization(),
    Conv1D(64, 3, activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),

    # Block 2
    Conv1D(128, 3, activation='relu', padding='same'),
    BatchNormalization(),
    Conv1D(128, 3, activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),

    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.4),
    Dense(1, activation='sigmoid')
])

In [ ]:
#Compiling
cnn_smote_isfraud.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy', Precision(name='precision'), Recall(name='recall')]
)


# Fitting with early stopping
cnn_smote_isfraud.fit(
    X_train_smote_cnn, y_train_smote,
    epochs=30,
    batch_size=64,
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=1
)

# Predicting & evaluation
y_pred_probs_smote_cnn_isfraud = cnn_smote_isfraud.predict(X_test_cnn).flatten()
y_pred_smote_cnn_isfraud = (y_pred_probs_smote_cnn_isfraud > 0.5).astype(int)

cm_smote_cnn_isfraud = confusion_matrix(y_test, y_pred_smote_cnn_isfraud)
roc_smote_cnn_isfraud = roc_auc_score(y_test, y_pred_probs_smote_cnn_isfraud)
cm_smote_cnn_isfraud

In [ ]:
# Saving the model
model_dir = "/content/drive/My Drive/Thesis Project"
os.makedirs(model_dir, exist_ok=True)
save_path = os.path.join(model_dir, "cnn_smote_ecommerce.h5")
cnn_smote_isfraud.save(save_path)
print("E-com SMOTE-CNN saved to:", save_path)

REALTIME INFERENCES

In [ ]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt

#Loading model
model_path = "/content/drive/My Drive/Thesis Project/cnn_smote_ecommerce.h5"
cnn_smote = load_model(model_path)

X_test_np = X_test.values
y_test_np = y_test.values

if X_test_np.ndim == 2:
    X_test_np = np.expand_dims(X_test_np, -1)

fraud_idx = np.where(y_test_np == 1)[0]
assert fraud_idx.size > 0, "No fraud rows in test set."
fraud_pool_X = X_test_np[fraud_idx]


#Simulation settings
domain_name  = "e-commerce"
lambda_fraud = 60.6
runs         = 10000
threshold    = 0.50
rng = np.random.default_rng(42)


# Monte Carlo
arrivals_F   = rng.poisson(lam=lambda_fraud, size=runs)
tp_vec       = np.zeros(runs, dtype=int)
det_rate_pct = np.full(runs, np.nan)

for r in range(runs):
    F = arrivals_F[r]
    if F == 0:
        continue
    s_idx = rng.choice(fraud_pool_X.shape[0], size=F, replace=True)
    X_r   = fraud_pool_X[s_idx]

    # Predicting probabilities
    p_r  = cnn_smote.predict(X_r, verbose=0).reshape(-1)
    yhat = (p_r >= threshold).astype(int)

    TP = int(yhat.sum())
    tp_vec[r]       = TP
    det_rate_pct[r] = 100.0 * TP / F

mc_df   = pd.DataFrame({"run": np.arange(1, runs+1),
                        "fraud_arrivals": arrivals_F,
                        "tp": tp_vec,
                        "detection_pct": det_rate_pct})
plot_df = mc_df.dropna(subset=["detection_pct"])


In [ ]:
#Summary table
summary_table = pd.DataFrame([{
    "model": "CNN (SMOTE)",
    "threshold": threshold,
    "mean_detection_pct": plot_df["detection_pct"].mean(),
    "median_detection_pct": plot_df["detection_pct"].median(),
    "p05_detection_pct": np.percentile(plot_df["detection_pct"], 5),
    "p95_detection_pct": np.percentile(plot_df["detection_pct"], 95),
    "mean_tp_per_sec": lambda_fraud * plot_df["detection_pct"].mean() / 100.0,
    "p05_tp_per_sec":  lambda_fraud * np.percentile(plot_df["detection_pct"], 5)  / 100.0,
    "p95_tp_per_sec":  lambda_fraud * np.percentile(plot_df["detection_pct"], 95) / 100.0
}])

print(summary_table.to_string(index=False))

In [ ]:
# Histogram
plt.figure(figsize=(8,5))
plt.hist(plot_df["detection_pct"], bins=40)
plt.title(f"Detection rate per second — Monte Carlo ({domain_name})")
plt.xlabel("Detection Rate per second (%)")
plt.ylabel("Count of runs")
plt.tight_layout()
plt.show()


In [ ]:
# ECDF: y goes 0..1 by definition
x = np.sort(plot_df["detection_pct"].values)
y = np.arange(1, x.size+1) / x.size
plt.figure(figsize=(8,5))
plt.plot(x, y)
plt.title(f"ECDF of per-second detection — Monte Carlo ({domain_name})")
plt.xlabel("Detection Rate per second (%)")
plt.ylabel("ECDF")
plt.ylim(0, 1)
plt.tight_layout()
plt.show()

In [ ]:
# Real-time Monte Carlo — CNN (e-commerce, SMOTE)
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.metrics import roc_auc_score

# Loading model + test data
model_path = "/content/drive/My Drive/Thesis Project/cnn_smote_ecommerce.h5"
cnn_smote = load_model(model_path)

# If X_test / y_test are pandas objects:
X_test_np = X_test.values if hasattr(X_test, "values") else np.asarray(X_test)
y_test_np = y_test.values if hasattr(y_test, "values") else np.asarray(y_test)

y_test_np = y_test_np.astype(int)
assert set(np.unique(y_test_np)).issubset({0,1}), "y_test must be binary {0,1}"
assert X_test_np.shape[0] == y_test_np.shape[0]

# If your CNN expects
def _ensure_cnn_shape(model, X):
    in_shape = model.input_shape
    if X.ndim == 2 and len(in_shape) == 3:
        return np.expand_dims(X, axis=-1)
    return X

X_test_np = _ensure_cnn_shape(cnn_smote, X_test_np)


# Simulation settings
lambda_per_sec = 60.6
runs           = 10000
thr_cnn        = 0.50
rng            = np.random.default_rng(123)

In [ ]:
# Probability helper
def predict_prob_keras(model, X_batch, batch_size=4096):
    """Return p(y=1) for a Keras model (handles (n,1) or (n,) outputs)."""
    p = model.predict(X_batch, batch_size=batch_size, verbose=0)
    return np.asarray(p).reshape(-1)


In [ ]:
# 4) Windowed Monte Carlo simulator
def simulate_realtime_fullpool_keras(model, threshold, lambda_rate_per_sec, runs,
                                     X_pool, y_pool, window_secs=1):
    """
    Each run is a window of length `window_secs`.
    arrivals ~ Poisson(lambda_rate_per_sec * window_secs)
    """
    N = X_pool.shape[0]
    arrivals = rng.poisson(lam=lambda_rate_per_sec * window_secs, size=runs)

    TP = np.zeros(runs, dtype=np.int32)
    FP = np.zeros(runs, dtype=np.int32)
    TN = np.zeros(runs, dtype=np.int32)
    FN = np.zeros(runs, dtype=np.int32)

    precision = np.full(runs, np.nan)
    recall    = np.full(runs, np.nan)
    accuracy  = np.full(runs, np.nan)
    f1        = np.full(runs, np.nan)
    auc_vec   = np.full(runs, np.nan)

    for r in range(runs):
        F = arrivals[r]
        if F == 0:
            continue

        idx  = rng.integers(0, N, size=F, dtype=np.int64)
        X_r  = X_pool[idx]
        y_r  = y_pool[idx]
        p_r  = predict_prob_keras(model, X_r)
        yhat = (p_r >= threshold).astype(np.int32)

        tp = np.sum((yhat == 1) & (y_r == 1))
        fp = np.sum((yhat == 1) & (y_r == 0))
        tn = np.sum((yhat == 0) & (y_r == 0))
        fn = np.sum((yhat == 0) & (y_r == 1))

        TP[r], FP[r], TN[r], FN[r] = tp, fp, tn, fn

        if (tp + fp) > 0:
            precision[r] = tp / (tp + fp)
        if (tp + fn) > 0:
            recall[r] = tp / (tp + fn)

        accuracy[r] = (tp + tn) / F
        if not np.isnan(precision[r]) and not np.isnan(recall[r]) and (precision[r] + recall[r]) > 0:
            f1[r] = 2 * precision[r] * recall[r] / (precision[r] + recall[r])

        # AUC only if both classes appear in the window
        if (y_r.min() == 0) and (y_r.max() == 1):
            try:
                auc_vec[r] = roc_auc_score(y_r, p_r)
            except Exception:
                pass

    mc_df = pd.DataFrame({
        "run": np.arange(1, runs+1, dtype=np.int32),
        "window_secs": window_secs,
        "arrivals": arrivals,
        "TP": TP, "FP": FP, "TN": TN, "FN": FN,
        "precision": precision, "recall": recall, "accuracy": accuracy, "f1": f1,
        "auc": auc_vec
    })

    summary = {
        "runs": runs,
        "window_secs": window_secs,
        "runs_with_tx": int(np.sum(arrivals > 0)),
        "mean_TP": float(np.nanmean(TP)),
        "mean_FP": float(np.nanmean(FP)),
        "mean_TN": float(np.nanmean(TN)),
        "mean_FN": float(np.nanmean(FN)),
        "mean_precision": float(np.nanmean(precision)),
        "mean_recall": float(np.nanmean(recall)),
        "mean_accuracy": float(np.nanmean(accuracy)),
        "mean_f1": float(np.nanmean(f1)),
        "mean_auc": float(np.nanmean(auc_vec)),
        "median_auc": float(np.nanmedian(auc_vec)),
        "n_auc_runs": int(np.sum(~np.isnan(auc_vec))),
    }
    summary_df = pd.DataFrame([summary])
    return {"mc_df": mc_df, "summary": summary_df}

In [ ]:
#Run CNN at 1s and 10s windows
res_cnn_1s  = simulate_realtime_fullpool_keras(
    model=cnn_smote, threshold=thr_cnn, lambda_rate_per_sec=lambda_per_sec,
    runs=runs, X_pool=X_test_np, y_pool=y_test_np, window_secs=1
)
res_cnn_10s = simulate_realtime_fullpool_keras(
    model=cnn_smote, threshold=thr_cnn, lambda_rate_per_sec=lambda_per_sec,
    runs=runs, X_pool=X_test_np, y_pool=y_test_np, window_secs=10
)

In [ ]:
# Comparing summaries
summary_table = pd.concat([
    res_cnn_1s["summary"].assign(model="CNN (SMOTE, e-comm)", time_unit="1 sec",  threshold=thr_cnn),
    res_cnn_10s["summary"].assign(model="CNN (SMOTE, e-comm)", time_unit="10 sec", threshold=thr_cnn),
], ignore_index=True)[[
    "model","time_unit",
    "mean_accuracy","mean_precision","mean_recall","mean_f1",
    "mean_auc"
]]

print(summary_table.to_string(index=False))

SHAP ANALYSIS FOR CNN SMOTE

In [ ]:
#Using same feature names
feature_names = X_train.columns.tolist()

#Sample from test and background data
background = X_train_smote_cnn[np.random.choice(X_train_smote_cnn.shape[0], 100, replace=False)]
X_explain = X_test_cnn[np.random.choice(X_test_cnn.shape[0], 50, replace=False)]

#Flattening for SHAP
background_flat = background.reshape(background.shape[0], -1)
X_explain_flat = X_explain.reshape(X_explain.shape[0], -1)

#Prediction wrapper for SMOTE-CNN model
def cnn_smote_predict(x_flat):
    x_reshaped = x_flat.reshape((-1, X_train_smote_cnn.shape[1], 1))
    return cnn_smote_isfraud.predict(x_reshaped).flatten()

#SHAP KernelExplainer
explainer = shap.KernelExplainer(cnn_smote_predict, background_flat)

#SHAP value computation
shap_values = explainer.shap_values(X_explain_flat)

In [ ]:
#SHAP Summary Plot
plt.figure(figsize=(6, 6))
shap.summary_plot(
    shap_values,
    X_explain_flat,
    feature_names=feature_names,
    plot_type="dot"
)
plt.title("SHAP Summary Plot - CNN (SMOTE)", fontsize=10)
plt.tight_layout()
plt.savefig("SHAP-CNN-SMOTE.png", dpi=300, bbox_inches='tight')

ADASYN + CNN¶

In [ ]:
adas = ADASYN(random_state=42)
X_train_adas, y_train_adas = adas.fit_resample(X_train, y_train)
X_train_adas_cnn = X_train_adas.values.reshape((X_train_adas.shape[0], X_train_adas.shape[1], 1))

# Early stopping callback
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True,
    verbose=1
)

#CNN architecture for ADASYN
cnn_adas_isfraud = Sequential([
    # Block 1
    Conv1D(32, 3, activation='relu', padding='same', input_shape=(X_train_adas_cnn.shape[1], 1)),
    BatchNormalization(),
    Conv1D(64, 3, activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),

    # Block 2
    Conv1D(128, 3, activation='relu', padding='same'),
    BatchNormalization(),
    Conv1D(128, 3, activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),

    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.4),
    Dense(1, activation='sigmoid')
])

In [ ]:
# Compiling
cnn_adas_isfraud.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy', Precision(name='precision'), Recall(name='recall')]
)


# Fitting with early stopping
cnn_adas_isfraud.fit(
    X_train_adas_cnn, y_train_adas,
    epochs=30,
    batch_size=64,
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=1
)


# Predicting & evaluation
y_pred_probs_adas_cnn_isfraud = cnn_adas_isfraud.predict(X_test_cnn).flatten()
y_pred_adas_cnn_isfraud = (y_pred_probs_adas_cnn_isfraud > 0.5).astype(int)

cm_adasyn_cnn_isfraud = confusion_matrix(y_test, y_pred_adas_cnn_isfraud)
roc_adasyn_cnn_isfraud = roc_auc_score(y_test, y_pred_probs_adas_cnn_isfraud)
cm_adasyn_cnn_isfraud

SHAP ANALYSIS FOR CNN ADASYN

In [ ]:
#Feature names
feature_names = X_train.columns.tolist()

#Sample background and test data
background = X_train_adas_cnn[np.random.choice(X_train_adas_cnn.shape[0], 100, replace=False)]
X_explain = X_test_cnn[np.random.choice(X_test_cnn.shape[0], 50, replace=False)]

#Flattening for SHAP KernelExplainer
background_flat = background.reshape(background.shape[0], -1)
X_explain_flat = X_explain.reshape(X_explain.shape[0], -1)

#Defining prediction wrapper for ADASYN model
def cnn_adas_predict(x_flat):
    x_reshaped = x_flat.reshape((-1, X_train_adas_cnn.shape[1], 1))
    return cnn_adas_isfraud.predict(x_reshaped).flatten()

#SHAP Explainer
explainer = shap.KernelExplainer(cnn_adas_predict, background_flat)

#Computing SHAP values
shap_values = explainer.shap_values(X_explain_flat)

In [ ]:
#SHAP Summary Plot
plt.figure(figsize=(6, 6))
shap.summary_plot(
    shap_values,
    X_explain_flat,
    feature_names=feature_names,
    plot_type="dot"
)
plt.title("SHAP Summary Plot - CNN (ADASYN)", fontsize=12)
plt.tight_layout()

Cost-Sensitive CNN

In [ ]:
class_weights_array = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = {0: class_weights_array[0], 1: class_weights_array[1]}

In [ ]:
#Early stopping callback
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True,
    verbose=1
)

#CNN architecture
cnn_cost_isfraud = Sequential([
    # Block 1
    Conv1D(32, 3, activation='relu', padding='same', input_shape=(X_train_cnn.shape[1], 1)),
    BatchNormalization(),
    Conv1D(64, 3, activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),

    # Block 2
    Conv1D(128, 3, activation='relu', padding='same'),
    BatchNormalization(),
    Conv1D(128, 3, activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),

    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.4),
    Dense(1, activation='sigmoid')
])

In [ ]:
#Compiling
cnn_cost_isfraud.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy', Precision(name='precision'), Recall(name='recall')]
)


# Fitting with early stopping and class weights
cnn_cost_isfraud.fit(
    X_train_cnn, y_train,
    epochs=30,
    batch_size=64,
    validation_split=0.2,
    class_weight=class_weights_dict,
    callbacks=[early_stop],
    verbose=1
)


# Predicting & evaluation
y_pred_probs_cost_cnn_isfraud = cnn_cost_isfraud.predict(X_test_cnn).flatten()
y_pred_cost_cnn_isfraud = (y_pred_probs_cost_cnn_isfraud > 0.5).astype(int)

cm_cost_cnn_isfraud = confusion_matrix(y_test, y_pred_cost_cnn_isfraud)
roc_cost_cnn_isfraud = roc_auc_score(y_test, y_pred_probs_cost_cnn_isfraud)
cm_cost_cnn_isfraud

SHAP ANALYSIS FOR CNN COST

In [ ]:
#Extract original feature names
feature_names = X_train.columns.tolist()

#Sample background and test data
background = X_train_cnn[np.random.choice(X_train_cnn.shape[0], 100, replace=False)]
X_explain = X_test_cnn[np.random.choice(X_test_cnn.shape[0], 50, replace=False)]

#Flattening inputs for SHAP KernelExplainer
background_flat = background.reshape(background.shape[0], -1)
X_explain_flat = X_explain.reshape(X_explain.shape[0], -1)

#Define prediction function for SHAP
def cnn_cost_predict(x_flat):
    x_reshaped = x_flat.reshape((-1, X_train_cnn.shape[1], 1))
    return cnn_cost_isfraud.predict(x_reshaped).flatten()

#Create SHAP explainer
explainer = shap.KernelExplainer(cnn_cost_predict, background_flat)

#Compute SHAP values
shap_values = explainer.shap_values(X_explain_flat)

In [ ]:
#SHAP Summary Plot
plt.figure(figsize=(6, 6))
shap.summary_plot(
    shap_values,
    X_explain_flat,
    feature_names=feature_names,
    plot_type="dot"
)
plt.title("SHAP Summary Plot - CNN (Cost-Sensitive)", fontsize=10)
plt.tight_layout()

Plot ROC Curves for CNN (E-commerce)

In [ ]:
fpr_base, tpr_base, _ = roc_curve(y_test, y_pred_probs_baseline_cnn_isfraud)
fpr_smote, tpr_smote, _ = roc_curve(y_test, y_pred_probs_smote_cnn_isfraud)
fpr_adas, tpr_adas, _ = roc_curve(y_test, y_pred_probs_adas_cnn_isfraud)
fpr_cost, tpr_cost, _ = roc_curve(y_test, y_pred_probs_cost_cnn_isfraud)

plt.figure(figsize=(8,6))
plt.plot(fpr_base, tpr_base, label="Baseline")
plt.plot(fpr_smote, tpr_smote, label="SMOTE")
plt.plot(fpr_adas, tpr_adas, label="ADASYN")
plt.plot(fpr_cost, tpr_cost, label="Cost-sensitive")
plt.plot([0,1],[0,1],'k--')
plt.title("ROC Curves for CNN - E-commerce Fraud")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend()
plt.grid()
plt.show()


Compare CNN Model Metrics

In [ ]:
results_cnn_isfraud = pd.DataFrame({
    "Method": ["Baseline", "SMOTE", "ADASYN", "Cost-sensitive"],
    "Accuracy": [
        accuracy_score(y_test, y_pred_baseline_cnn_isfraud),
        accuracy_score(y_test, y_pred_smote_cnn_isfraud),
        accuracy_score(y_test, y_pred_adas_cnn_isfraud),
        accuracy_score(y_test, y_pred_cost_cnn_isfraud)
    ],
    "Precision": [
        precision_score(y_test, y_pred_baseline_cnn_isfraud),
        precision_score(y_test, y_pred_smote_cnn_isfraud),
        precision_score(y_test, y_pred_adas_cnn_isfraud),
        precision_score(y_test, y_pred_cost_cnn_isfraud)
    ],
    "Recall": [
        recall_score(y_test, y_pred_baseline_cnn_isfraud),
        recall_score(y_test, y_pred_smote_cnn_isfraud),
        recall_score(y_test, y_pred_adas_cnn_isfraud),
        recall_score(y_test, y_pred_cost_cnn_isfraud)
    ],
    "F1": [
        f1_score(y_test, y_pred_baseline_cnn_isfraud),
        f1_score(y_test, y_pred_smote_cnn_isfraud),
        f1_score(y_test, y_pred_adas_cnn_isfraud),
        f1_score(y_test, y_pred_cost_cnn_isfraud)
    ],
    "AUC": [
        roc_baseline_cnn_isfraud,
        roc_smote_cnn_isfraud,
        roc_adasyn_cnn_isfraud,
        roc_cost_cnn_isfraud
    ]

})

print(results_cnn_isfraud)

# LSTM MODEL

BASELINE LSTM

In [ ]:
# Reshaping for LSTM [samples, timesteps, features]
X_train_lstm = X_train.values.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test_lstm  = X_test.values.reshape((X_test.shape[0], X_test.shape[1], 1))

In [ ]:
# Defining EarlyStopping callback
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True,
    verbose=1
)

# LSTM model
lstm_baseline = Sequential([
    # LSTM Block 1
    LSTM(128, input_shape=(X_train_lstm.shape[1], 1),
         return_sequences=True, dropout=0.3, recurrent_dropout=0.1),
    BatchNormalization(),

    # LSTM Block 2
    LSTM(128, return_sequences=True, dropout=0.3, recurrent_dropout=0.1),
    BatchNormalization(),

    # LSTM Block 3 (final, no return_sequences)
    LSTM(64, return_sequences=False, dropout=0.3, recurrent_dropout=0.1),
    BatchNormalization(),

    # Dense head
    Dense(256, activation='relu'),
    Dropout(0.4),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

In [ ]:
# Compiling model
lstm_baseline.compile(
    optimizer=Adam(learning_rate=0.0005),
    loss='binary_crossentropy',
    metrics=['accuracy', Precision(name='precision'), Recall(name='recall')]
)


# Fitting with early stopping and validation
lstm_baseline.fit(
    X_train_lstm, y_train,
    epochs=30,
    batch_size=64,
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=1
)

# Predictions
y_pred_probs_baseline_lstm = lstm_baseline.predict(X_test_lstm).flatten()
y_pred_baseline_lstm = (y_pred_probs_baseline_lstm > 0.5).astype(int)

In [ ]:
# Metrics
cm_baseline_lstm_isfraud = confusion_matrix(y_test, y_pred_baseline_lstm)
roc_baseline_lstm_isfraud = roc_auc_score(y_test, y_pred_probs_baseline_lstm)
cm_baseline_lstm_isfraud

SMOTE + LSTM

In [ ]:
# SMOTE
smote = SMOTE(random_state=42)
X_train_smote_lstm, y_train_smote_lstm = smote.fit_resample(X_train, y_train)
X_train_smote_lstm = X_train_smote_lstm.values.reshape((X_train_smote_lstm.shape[0], X_train_smote_lstm.shape[1], 1))

In [ ]:
# Early stopping callback
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True,
    verbose=1
)

# LSTM model for SMOTE
lstm_smote = Sequential([
    # LSTM Block 1
    LSTM(128, input_shape=(X_train_smote_lstm.shape[1], 1),
         return_sequences=True, dropout=0.3, recurrent_dropout=0.1),
    BatchNormalization(),

    # LSTM Block 2
    LSTM(128, return_sequences=True, dropout=0.3, recurrent_dropout=0.1),
    BatchNormalization(),

    # LSTM Block 3
    LSTM(64, return_sequences=False, dropout=0.3, recurrent_dropout=0.1),
    BatchNormalization(),

    # Dense head
    Dense(256, activation='relu'),
    Dropout(0.4),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

In [ ]:
# Compiling model
lstm_smote.compile(
    optimizer=Adam(learning_rate=0.0005),
    loss='binary_crossentropy',
    metrics=['accuracy', Precision(name='precision'), Recall(name='recall')]
)

# Fitting model with early stopping and validation split
lstm_smote.fit(
    X_train_smote_lstm, y_train_smote_lstm,
    epochs=30,
    batch_size=64,
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=1
)

# Predictions
y_pred_probs_smote_lstm = lstm_smote.predict(X_test_lstm).flatten()
y_pred_smote_lstm = (y_pred_probs_smote_lstm > 0.5).astype(int)

In [ ]:
# Metrics
cm_smote_lstm_isfraud = confusion_matrix(y_test, y_pred_smote_lstm)
roc_smote_lstm_isfraud = roc_auc_score(y_test, y_pred_probs_smote_lstm)
cm_smote_lstm_isfraud

ADASYN + LSTM

In [ ]:
# ADASYN
adas = ADASYN(random_state=42)
X_train_adas_lstm, y_train_adas_lstm = adas.fit_resample(X_train, y_train)
X_train_adas_lstm = X_train_adas_lstm.values.reshape((X_train_adas_lstm.shape[0], X_train_adas_lstm.shape[1], 1))

In [ ]:
# Early stopping setup
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True,
    verbose=1
)

# Defining model for ADASYN
lstm_adas = Sequential([
    # LSTM Block 1
    LSTM(128, input_shape=(X_train_adas_lstm.shape[1], 1),
         return_sequences=True, dropout=0.3, recurrent_dropout=0.1),
    BatchNormalization(),

    # LSTM Block 2
    LSTM(128, return_sequences=True, dropout=0.3, recurrent_dropout=0.1),
    BatchNormalization(),

    # LSTM Block 3
    LSTM(64, return_sequences=False, dropout=0.3, recurrent_dropout=0.1),
    BatchNormalization(),

    # Dense head
    Dense(256, activation='relu'),
    Dropout(0.4),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

In [ ]:
# Compiling
lstm_adas.compile(
    optimizer=Adam(learning_rate=0.0005),
    loss='binary_crossentropy',
    metrics=['accuracy', Precision(name='precision'), Recall(name='recall')]
)

# Training with validation and early stopping
lstm_adas.fit(
    X_train_adas_lstm, y_train_adas_lstm,
    epochs=30,
    batch_size=64,
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=1
)

# Predicting and evaluation
y_pred_probs_adas_lstm = lstm_adas.predict(X_test_lstm).flatten()
y_pred_adas_lstm = (y_pred_probs_adas_lstm > 0.5).astype(int)

In [ ]:
# Metrics
cm_adasyn_lstm_isfraud = confusion_matrix(y_test, y_pred_adas_lstm)
roc_adasyn_lstm_isfraud = roc_auc_score(y_test, y_pred_probs_adas_lstm)
cm_adasyn_lstm_isfraud

Cost-Sensitive LSTM

In [ ]:
# Early stopping setup
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True,
    verbose=1
)

# LSTM model for Cost Sensitive
lstm_cost = Sequential([
    # LSTM Block 1
    LSTM(128, input_shape=(X_train_lstm.shape[1], 1),
         return_sequences=True, dropout=0.3, recurrent_dropout=0.1),
    BatchNormalization(),

    # LSTM Block 2
    LSTM(128, return_sequences=True, dropout=0.3, recurrent_dropout=0.1),
    BatchNormalization(),

    # LSTM Block 3
    LSTM(64, return_sequences=False, dropout=0.3, recurrent_dropout=0.1),
    BatchNormalization(),

    # Dense head
    Dense(256, activation='relu'),
    Dropout(0.4),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

In [ ]:
# Compiling
lstm_cost.compile(
    optimizer=Adam(learning_rate=0.0005),
    loss='binary_crossentropy',
    metrics=['accuracy', Precision(name='precision'), Recall(name='recall')]
)

# Fitting model with class weights and early stopping
lstm_cost.fit(
    X_train_lstm, y_train,
    epochs=30,
    batch_size=64,
    class_weight=class_weights_dict,
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=1
)

# Predictions
y_pred_probs_cost_lstm = lstm_cost.predict(X_test_lstm).flatten()
y_pred_cost_lstm = (y_pred_probs_cost_lstm > 0.5).astype(int)

In [ ]:
# Metrics
cm_cost_lstm_isfraud = confusion_matrix(y_test, y_pred_cost_lstm)
roc_cost_lstm_isfraud = roc_auc_score(y_test, y_pred_probs_cost_lstm)
cm_cost_lstm_isfraud

Plot ROC Curves

In [ ]:
fpr_base, tpr_base, _ = roc_curve(y_test, y_pred_probs_baseline_lstm)
fpr_smote, tpr_smote, _ = roc_curve(y_test, y_pred_probs_smote_lstm)
fpr_adas, tpr_adas, _ = roc_curve(y_test, y_pred_probs_adas_lstm)
fpr_cost, tpr_cost, _ = roc_curve(y_test, y_pred_probs_cost_lstm)

plt.figure(figsize=(8,6))
plt.plot(fpr_base, tpr_base, label="Baseline")
plt.plot(fpr_smote, tpr_smote, label="SMOTE")
plt.plot(fpr_adas, tpr_adas, label="ADASYN")
plt.plot(fpr_cost, tpr_cost, label="Cost-sensitive")
plt.plot([0,1],[0,1],'k--')
plt.title("ROC Curves for LSTM - E-commerce Fraud")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend()
plt.grid()
plt.show()

Model Comparison Table

In [ ]:
results_lstm_isfraud = pd.DataFrame({
    "Method": ["Baseline", "SMOTE", "ADASYN", "Cost-sensitive"],
    "Accuracy": [
        accuracy_score(y_test, y_pred_baseline_lstm),
        accuracy_score(y_test, y_pred_smote_lstm),
        accuracy_score(y_test, y_pred_adas_lstm),
        accuracy_score(y_test, y_pred_cost_lstm)
    ],
    "Precision": [
        precision_score(y_test, y_pred_baseline_lstm),
        precision_score(y_test, y_pred_smote_lstm),
        precision_score(y_test, y_pred_adas_lstm),
        precision_score(y_test, y_pred_cost_lstm)
    ],
    "Recall": [
        recall_score(y_test, y_pred_baseline_lstm),
        recall_score(y_test, y_pred_smote_lstm),
        recall_score(y_test, y_pred_adas_lstm),
        recall_score(y_test, y_pred_cost_lstm)
    ],
    "F1": [
        f1_score(y_test, y_pred_baseline_lstm),
        f1_score(y_test, y_pred_smote_lstm),
        f1_score(y_test, y_pred_adas_lstm),
        f1_score(y_test, y_pred_cost_lstm)
    ],
    "AUC": [
        roc_baseline_lstm_isfraud,
        roc_smote_lstm_isfraud,
        roc_adasyn_lstm_isfraud,
        roc_cost_lstm_isfraud
    ]
})

print(results_lstm_isfraud)

# HYBRID MODEL

Baseline CNN + LSTM

In [ ]:
# Reshaping input
X_train_cnn_lstm = X_train.values.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test_cnn_lstm  = X_test.values.reshape((X_test.shape[0], X_test.shape[1], 1))

In [ ]:
# Early stopping
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True,
    verbose=1
)

# Define enhanced CNN+LSTM model
cnn_lstm_baseline = Sequential([
    # Conv Block 1
    Conv1D(32, 3, activation='relu', padding='same', input_shape=(X_train_cnn_lstm.shape[1], 1)),
    BatchNormalization(),
    Conv1D(64, 3, activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),

    # Conv Block 2
    Conv1D(128, 3, activation='relu', padding='same'),
    BatchNormalization(),
    Conv1D(128, 3, activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),

    # LSTM stack
    LSTM(128, return_sequences=True, dropout=0.3, recurrent_dropout=0.1),
    BatchNormalization(),
    LSTM(64, return_sequences=False, dropout=0.3, recurrent_dropout=0.1),
    BatchNormalization(),

    # Dense head
    Dense(128, activation='relu'),
    Dropout(0.4),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

In [ ]:
# Compiling
cnn_lstm_baseline.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy', Precision(name='precision'), Recall(name='recall')]
)

# Fitting with early stopping
cnn_lstm_baseline.fit(
    X_train_cnn_lstm, y_train,
    epochs=30,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=1
)

# Predicting & evaluation
y_pred_probs_baseline_cnn_lstm = cnn_lstm_baseline.predict(X_test_cnn_lstm).flatten()
y_pred_baseline_cnn_lstm = (y_pred_probs_baseline_cnn_lstm > 0.5).astype(int)

In [ ]:
cm_baseline_cnn_lstm_isfraud = confusion_matrix(y_test, y_pred_baseline_cnn_lstm)
roc_baseline_cnn_lstm_isfraud = roc_auc_score(y_test, y_pred_probs_baseline_cnn_lstm)
cm_baseline_cnn_lstm_isfraud

SMOTE + CNN + LSTM

In [ ]:
X_train_smote_cnn_lstm = X_train_smote.values.reshape((X_train_smote.shape[0], X_train_smote.shape[1], 1))

In [ ]:
# Early stopping
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True,
    verbose=1
)

# Defining enhanced CNN+LSTM model
cnn_lstm_smote = Sequential([
    # Conv Block 1
    Conv1D(32, 3, activation='relu', padding='same',
           input_shape=(X_train_smote_cnn_lstm.shape[1], 1)),
    BatchNormalization(),
    Conv1D(64, 3, activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),

    # Conv Block 2
    Conv1D(128, 3, activation='relu', padding='same'),
    BatchNormalization(),
    Conv1D(128, 3, activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),

    # LSTM stack
    LSTM(128, return_sequences=True, dropout=0.3, recurrent_dropout=0.1),
    BatchNormalization(),
    LSTM(64, return_sequences=False, dropout=0.3, recurrent_dropout=0.1),
    BatchNormalization(),

    # Dense head
    Dense(128, activation='relu'),
    Dropout(0.4),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

In [ ]:
# Compiling model
cnn_lstm_smote.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy', Precision(name='precision'), Recall(name='recall')]
)

# Fitting with validation and early stopping
cnn_lstm_smote.fit(
    X_train_smote_cnn_lstm, y_train_smote,
    epochs=30,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=1
)

# Predicting & evaluation
y_pred_probs_smote_cnn_lstm = cnn_lstm_smote.predict(X_test_cnn_lstm).flatten()
y_pred_smote_cnn_lstm = (y_pred_probs_smote_cnn_lstm > 0.5).astype(int)

In [ ]:
cm_smote_cnn_lstm_isfraud = confusion_matrix(y_test, y_pred_smote_cnn_lstm)
roc_smote_cnn_lstm_isfraud = roc_auc_score(y_test, y_pred_probs_smote_cnn_lstm)
cm_smote_cnn_lstm_isfraud

ADASYN + CNN + LSTM

In [ ]:
X_train_adas_cnn_lstm = X_train_adas.values.reshape((X_train_adas.shape[0], X_train_adas.shape[1], 1))

In [ ]:
# Early stopping
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True,
    verbose=1
)

# Defining enhanced CNN+LSTM model ADASYN
cnn_lstm_adas = Sequential([
    # Conv Block 1
    Conv1D(32, 3, activation='relu', padding='same',
           input_shape=(X_train_adas_cnn_lstm.shape[1], 1)),
    BatchNormalization(),
    Conv1D(64, 3, activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),

    # Conv Block 2
    Conv1D(128, 3, activation='relu', padding='same'),
    BatchNormalization(),
    Conv1D(128, 3, activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),

    # LSTM stack
    LSTM(128, return_sequences=True, dropout=0.3, recurrent_dropout=0.1),
    BatchNormalization(),
    LSTM(64, return_sequences=False, dropout=0.3, recurrent_dropout=0.1),
    BatchNormalization(),

    # Dense head
    Dense(128, activation='relu'),
    Dropout(0.4),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

In [ ]:
# Compiling model
cnn_lstm_adas.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy', Precision(name='precision'), Recall(name='recall')]
)

# Fitting with early stopping and validation split
cnn_lstm_adas.fit(
    X_train_adas_cnn_lstm, y_train_adas,
    epochs=30,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=1
)

# Predicting and evaluation
y_pred_probs_adas_cnn_lstm = cnn_lstm_adas.predict(X_test_cnn_lstm).flatten()
y_pred_adas_cnn_lstm = (y_pred_probs_adas_cnn_lstm > 0.5).astype(int)

In [ ]:
cm_adasyn_cnn_lstm_isfraud = confusion_matrix(y_test, y_pred_adas_cnn_lstm)
roc_adasyn_cnn_lstm_isfraud = roc_auc_score(y_test, y_pred_probs_adas_cnn_lstm)
cm_adasyn_cnn_lstm_isfraud

Cost-Sensitive CNN + LSTM

In [ ]:
# Early stopping
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True,
    verbose=1
)

# Define enhanced CNN+LSTM model for Cost-sensitive
cnn_lstm_cost = Sequential([
    # Conv Block 1
    Conv1D(32, 3, activation='relu', padding='same',
           input_shape=(X_train_cnn_lstm.shape[1], 1)),
    BatchNormalization(),
    Conv1D(64, 3, activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),

    # Conv Block 2 (added)
    Conv1D(128, 3, activation='relu', padding='same'),
    BatchNormalization(),
    Conv1D(128, 3, activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),

    # LSTM stack
    LSTM(128, return_sequences=True, dropout=0.3, recurrent_dropout=0.1),
    BatchNormalization(),
    LSTM(64, return_sequences=False, dropout=0.3, recurrent_dropout=0.1),
    BatchNormalization(),

    # Dense head
    Dense(128, activation='relu'),
    Dropout(0.4),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

In [ ]:
# Compiling model
cnn_lstm_cost.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy', Precision(name='precision'), Recall(name='recall')]
)

# Fitting model with class weights and early stopping
cnn_lstm_cost.fit(
    X_train_cnn_lstm, y_train,
    epochs=30,
    batch_size=32,
    validation_split=0.2,
    class_weight=class_weights_dict,
    callbacks=[early_stop],
    verbose=1
)

# Predicting and evaluation
y_pred_probs_cost_cnn_lstm = cnn_lstm_cost.predict(X_test_cnn_lstm).flatten()
y_pred_cost_cnn_lstm = (y_pred_probs_cost_cnn_lstm > 0.5).astype(int)

In [ ]:
cm_cost_cnn_lstm_isfraud = confusion_matrix(y_test, y_pred_cost_cnn_lstm)
roc_cost_cnn_lstm_isfraud = roc_auc_score(y_test, y_pred_probs_cost_cnn_lstm)
cm_cost_cnn_lstm_isfraud

ROC Curves for CNN + LSTM Models

In [ ]:
# ROC Curve Components
fpr_base, tpr_base, _ = roc_curve(y_test, y_pred_probs_baseline_cnn_lstm)
fpr_smote, tpr_smote, _ = roc_curve(y_test, y_pred_probs_smote_cnn_lstm)
fpr_adas, tpr_adas, _ = roc_curve(y_test, y_pred_probs_adas_cnn_lstm)
fpr_cost, tpr_cost, _ = roc_curve(y_test, y_pred_probs_cost_cnn_lstm)

# Plot ROC Curves
plt.figure(figsize=(8,6))
plt.plot(fpr_base, tpr_base, label="Baseline")
plt.plot(fpr_smote, tpr_smote, label="SMOTE")
plt.plot(fpr_adas, tpr_adas, label="ADASYN")
plt.plot(fpr_cost, tpr_cost, label="Cost-sensitive")
plt.plot([0,1],[0,1],'k--')
plt.title("ROC Curves for CNN + LSTM - E-commerce Fraud")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend()
plt.grid()
plt.show()

Model Metrics Comparison Table

In [ ]:
results_cnn_lstm_isfraud = pd.DataFrame({
    "Method": ["Baseline", "SMOTE", "ADASYN", "Cost-sensitive"],
    "Accuracy": [
        accuracy_score(y_test, y_pred_baseline_cnn_lstm),
        accuracy_score(y_test, y_pred_smote_cnn_lstm),
        accuracy_score(y_test, y_pred_adas_cnn_lstm),
        accuracy_score(y_test, y_pred_cost_cnn_lstm)
    ],
    "Precision": [
        precision_score(y_test, y_pred_baseline_cnn_lstm),
        precision_score(y_test, y_pred_smote_cnn_lstm),
        precision_score(y_test, y_pred_adas_cnn_lstm),
        precision_score(y_test, y_pred_cost_cnn_lstm)
    ],
    "Recall": [
        recall_score(y_test, y_pred_baseline_cnn_lstm),
        recall_score(y_test, y_pred_smote_cnn_lstm),
        recall_score(y_test, y_pred_adas_cnn_lstm),
        recall_score(y_test, y_pred_cost_cnn_lstm)
    ],
    "F1": [
        f1_score(y_test, y_pred_baseline_cnn_lstm),
        f1_score(y_test, y_pred_smote_cnn_lstm),
        f1_score(y_test, y_pred_adas_cnn_lstm),
        f1_score(y_test, y_pred_cost_cnn_lstm)
    ],
    "AUC": [
        roc_baseline_cnn_lstm_isfraud,
        roc_smote_cnn_lstm_isfraud,
        roc_adasyn_cnn_lstm_isfraud,
        roc_cost_cnn_lstm_isfraud
    ]
})

print(results_cnn_lstm_isfraud)